In [1]:
import re
import random 
import os
import numpy as np
data_path = "/home/parastoo/Downloads/human_text.txt"
data_path2 = "/home/parastoo/Downloads/robot_text.txt"
path = "/home/parastoo/Downloads/glove.6B(1)"

In [2]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [3]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from numpy import array
import pandas as pd

In [4]:
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
  lines2 = f.read().split('\n')

In [5]:
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    return txt

In [6]:
lines = [clean_text(line) for line in lines]
lines2 = [clean_text(line) for line in lines2]

# grouping lines by response pair
pairs = list(zip(lines,lines2))
#random.shuffle(pairs)

In [7]:
import pandas as pd
print(len(pd.DataFrame(pairs)))

2363


In [8]:
pd.DataFrame(pairs).head()

,0,1
0,start,hi there how are you
1,oh thanks i am fine this is an evening in my...,here is afternoon
2,how do you feel today tell me something abou...,my name is rdany but you can call me dany the ...
3,how many virtual friends have you got,i have many but not enough to fully understa...
4,is that forbidden for you to tell the exact nu...,i have talked with 143 users counting 7294 lin...


In [9]:
max_input_length = max([len(line) for line in lines])
max_target_length = max([len(line) for line in lines2])
print("max input length: ", max_input_length)
print("max target length: ", max_target_length)

max input length:  1266
max target length:  935


In [10]:
for i in range(len(lines2)):
    lines2[i] = ' '.join(lines2[i].split())

In [11]:
word2count = {}

for line in lines:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for line in lines2:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [12]:
thresh = 5

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= thresh:
        vocab[word] = word_num
        word_num += 1
        

In [13]:
for i in range(len(lines2)):
    lines2[i] = '<SOS> ' + lines2[i] + ' <EOS>'



tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1
    
    

vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0
lines2[:5]

['<SOS> hi there how are you <EOS>',
 '<SOS> here is afternoon <EOS>',
 '<SOS> my name is rdany but you can call me dany the r means robot i hope we can be virtual friends <EOS>',
 '<SOS> i have many but not enough to fully understand humans beings <EOS>',
 '<SOS> i have talked with 143 users counting 7294 lines of text <EOS>']

In [14]:
inv_vocab = {w:v for v, w in vocab.items()}
pd.DataFrame(inv_vocab.items()).head()

,0,1
0,0,<PAD>
1,1,oh
2,2,thanks
3,3,i
4,4,am


In [15]:
encoder_inp = []
for line in lines:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

decoder_inp = []
for line in lines2:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)

pd.DataFrame(decoder_inp).head()

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
0,915,94,311.0,13.0,40.0,15.0,913.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,915,230,7.0,914.0,913.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,915,11,87.0,7.0,134.0,81.0,15.0,53.0,302.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,915,3,26.0,23.0,81.0,54.0,561.0,30.0,873.0,371.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,915,3,26.0,122.0,47.0,914.0,700.0,914.0,914.0,914.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_inp = pad_sequences(encoder_inp, 50, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 50, padding='post', truncating='post')
pd.DataFrame(decoder_inp).head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,915,94,311,13,40,15,913,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,915,230,7,914,913,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,915,11,87,7,134,81,15,53,302,19,...,0,0,0,0,0,0,0,0,0,0
3,915,3,26,23,81,54,561,30,873,371,...,0,0,0,0,0,0,0,0,0,0
4,915,3,26,122,47,914,700,914,914,914,...,0,0,0,0,0,0,0,0,0,0


In [17]:
decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 
decoder_final_output = pad_sequences(decoder_final_output, 50, padding='post', truncating='post')
pd.DataFrame(decoder_final_output)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,94,311,13,40,15,913,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,230,7,914,913,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11,87,7,134,81,15,53,302,19,415,...,0,0,0,0,0,0,0,0,0,0
3,3,26,23,81,54,561,30,873,371,80,...,0,0,0,0,0,0,0,0,0,0
4,3,26,122,47,914,700,914,914,914,82,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2358,94,230,13,40,15,913,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2359,98,3,4,5,97,913,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2360,75,755,914,913,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2361,94,311,13,40,15,913,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
from tensorflow.keras.utils import to_categorical
decoder_final_output = to_categorical(decoder_final_output, len(vocab))

In [19]:
embeddings_index = {}
with open('/home/parastoo/Downloads/glove.6B(1)/glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print("Glove Loded!")

Glove Loded!


In [20]:
embedding_dimention = 50
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index)+1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix
embedding_matrix = embedding_matrix_creater(50, word_index=vocab)    


In [21]:
embedding_matrix.shape

(918, 50)

In [22]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Bidirectional, Concatenate, Dropout, Attention

In [23]:

VOCAB_SIZE = len(vocab)
MAX_LEN = 50

embed = Embedding(VOCAB_SIZE+1, 
                  50, 
                  
                  input_length=50,
                  trainable=True)

embed.build((None,))
embed.set_weights([embedding_matrix])


MODEL

In [24]:
enc_inp = Input(shape=(50, ))
#embed = Embedding(VOCAB_SIZE+1, 50, mask_zero=True, input_length=13)(enc_inp)
enc_embed = embed(enc_inp)
enc_lstm = Bidirectional(LSTM(400, return_state=True, dropout=0.05, return_sequences = True))

encoder_outputs, forward_h, forward_c, backward_h, backward_c = enc_lstm(enc_embed)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

enc_states = [state_h, state_c]


dec_inp = Input(shape=(50, ))
dec_embed = embed(dec_inp)
dec_lstm = LSTM(400*2, return_state=True, return_sequences=True, dropout=0.05)
output, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

# attention
attn_layer = AttentionLayer()
attn_op, attn_state = attn_layer([encoder_outputs, output])
decoder_concat_input = Concatenate(axis=-1)([output, attn_op])


dec_dense = Dense(VOCAB_SIZE, activation='softmax')
final_output = dec_dense(decoder_concat_input)

model = Model([enc_inp, dec_inp], final_output)
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

model.summary()

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 50)       45900       input_1[0][0]                    
                                                                 input_2[0][0]                    
______________________________________

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit([encoder_inp, decoder_inp], decoder_final_output, epochs=30
          , batch_size=24, validation_split=0.15)

Epoch 1/30
84/84 [==============================] - 81s 962ms/step - loss: 1.1325 - acc: 0.8149 - val_loss: 0.8535 - val_acc: 0.8534
Epoch 2/30
84/84 [==============================] - 81s 966ms/step - loss: 0.8467 - acc: 0.8472 - val_loss: 0.7212 - val_acc: 0.8724
Epoch 3/30
84/84 [==============================] - 81s 967ms/step - loss: 0.7745 - acc: 0.8567 - val_loss: 0.6885 - val_acc: 0.8773
Epoch 4/30
84/84 [==============================] - 82s 973ms/step - loss: 0.7228 - acc: 0.8621 - val_loss: 0.6526 - val_acc: 0.8817
Epoch 5/30
84/84 [==============================] - 81s 964ms/step - loss: 0.6789 - acc: 0.8658 - val_loss: 0.6419 - val_acc: 0.8824
Epoch 6/30
84/84 [==============================] - 81s 965ms/step - loss: 0.6374 - acc: 0.8689 - val_loss: 0.6378 - val_acc: 0.8814
Epoch 7/30
84/84 [==============================] - 81s 967ms/step - loss: 0.5974 - acc: 0.8726 - val_loss: 0.6311 - val_acc: 0.8845
Epoch 8/30
84/84 [==============================] - 82s 978ms/step - 

In [26]:
decoder_final_output.shape

(2363, 50, 917)

In [27]:
enc_model = tf.keras.models.Model(enc_inp, [encoder_outputs, enc_states])


decoder_state_input_h = tf.keras.layers.Input(shape=( 400 * 2,))
decoder_state_input_c = tf.keras.layers.Input(shape=( 400 * 2,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]

#decoder_output = dec_dense(decoder_outputs)

dec_model = tf.keras.models.Model([dec_inp, decoder_states_inputs],
                                      [decoder_outputs] + decoder_states)

In [ ]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
print("##########################################")
print("#       start chatting ver. 1.0          #")
print("##########################################")


prepro1 = ""
while prepro1 != 'q':
    
    prepro1 = input("you : ")
    try:
        prepro1 = clean_text(prepro1)
        prepro = [prepro1]
        
        txt = []
        for x in prepro:
            lst = []
            for y in x.split():
                try:
                    lst.append(vocab[y])
                except:
                    lst.append(vocab['<OUT>'])
            txt.append(lst)
        txt = pad_sequences(txt, 13, padding='post')


        ###
        enc_op, stat = enc_model.predict( txt )

        empty_target_seq = np.zeros( ( 1 , 1) )
        empty_target_seq[0, 0] = vocab['<SOS>']
        stop_condition = False
        decoded_translation = ''


        while not stop_condition :

            dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + stat )

            ###
            ###########################
            attn_op, attn_state = attn_layer([enc_op, dec_outputs])
            decoder_concat_input = Concatenate(axis=-1)([dec_outputs, attn_op])
            decoder_concat_input = dec_dense(decoder_concat_input)
            ###########################

            sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )

            sampled_word = inv_vocab[sampled_word_index] + ' '

            if sampled_word != '<EOS> ':
                decoded_translation += sampled_word           


            if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
                stop_condition = True

            empty_target_seq = np.zeros( ( 1 , 1 ) )  
            empty_target_seq[ 0 , 0 ] = sampled_word_index
            stat = [ h , c ] 

        print("chatbot attention : ", decoded_translation )
        print("==============================================")

    except:
        print("sorry didn't got you , please type again :( ")
ou




##########################################
#       start chatting ver. 1.0          #
##########################################
you : hi
chatbot attention :  hi how are you 
you : i am fine
chatbot attention :  good do you speak english 
you : yes i speak english and persian
chatbot attention :  nice quite <OUT> 
you : do you like coffee?
chatbot attention :  sure what is your favorite group 
you : how old are you
chatbot attention :  i am 22 <OUT> but <OUT> 
you : i am 21.
chatbot attention :  nice humans really love food 
you : yes i love food too
chatbot attention :  
you : what is yout name
chatbot attention :  special <OUT> <OUT> <OUT> to create something new 
you : what is your name
chatbot attention :  my name is rdany but you can call me dany 
you : do you like playing football ?
chatbot attention :  i am a robot <OUT> the world 
you : where are you from?
chatbot attention :  i am form a lab on argentina nice place when you get used to 


In [ ]:
# input_docs = []
# target_docs = []
# input_tokens = set()
# target_tokens = set()

# for line in pairs:
#   input_doc, target_doc = line[0], line[1]
#   input_docs.append(input_doc)
  
#   target_doc = '<START> ' + target_doc + ' <END>'
#   target_docs.append(target_doc)
  
#   for token in input_doc.split():
#     if token not in input_tokens:
#       input_tokens.add(token)

#   for token in target_doc.split():
#     if token not in target_tokens:
#       target_tokens.add(token)
# input_tokens = sorted(set(list(input_tokens)))
# target_tokens = sorted(set(list(target_tokens)))

# num_encoder_tokens = len(input_tokens)
# num_decoder_tokens = len(target_tokens)

# print(" num of input unique tokens ", num_encoder_tokens ,"\n num of target uniquie tokens" ,num_decoder_tokens)     

In [ ]:
# len(set(input_tokens + target_tokens))

In [ ]:
# input_features_dict = dict(
#     [(token, i) for i, token in enumerate(input_tokens)])
# target_features_dict = dict(
#     [(token, i) for i, token in enumerate(target_tokens)])
# reverse_input_features_dict = dict(
#     (i, token) for token, i in input_features_dict.items())
# reverse_target_features_dict = dict(
#     (i, token) for token, i in target_features_dict.items())

In [ ]:
# pd.DataFrame(input_features_dict.items(),columns={"word","index"})[20:25]

In [ ]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# VOCAB_SIZE = 14999
# tokenizer = Tokenizer(num_words=VOCAB_SIZE)
# tokenizer.fit_on_texts(input_docs + target_docs)
# dictionary = tokenizer.word_index

# input_seq = tokenizer.texts_to_sequences(input_docs)
# target_seq = tokenizer.texts_to_sequences(target_docs)

# #** fit on text starts the indexes from one not zero. which is good for padding
# pd.DataFrame(dictionary.items()).head()

In [ ]:
# word2idx = {}
# VOCAB_SIZE = 14999
# for k, v in dictionary.items():
#     if v < VOCAB_SIZE:
#         word2idx[k] = v
#     if v >= VOCAB_SIZE-1:
#         continue

# len(word2idx)
# idx2word = {}
# for k,v in word2idx.items():
#     idx2word[v] = k

In [ ]:
# idx2word[1]

In [ ]:
# pd.DataFrame(word2idx.items(),columns={"word","index"}).head()

In [ ]:
# pd.DataFrame(input_seq).head()

In [ ]:
# pd.DataFrame(target_seq).head()

In [ ]:
# #Maximum length of sentences in input and target documents
# max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
# max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])
# print("max length of an encoder seq " , max_encoder_seq_length)
# print("max length of an decoder seq ", max_decoder_seq_length)

In [ ]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# encoder_input_data = pad_sequences(input_seq, maxlen=max_encoder_seq_length, dtype='int32', padding='post', truncating='post')
# decoder_input_data = pad_sequences(target_seq, maxlen=max_decoder_seq_length, dtype='int32', padding='post', truncating='post')
# print("max dcoder input: ",pd.DataFrame(decoder_input_data).shape)
# print("max encoder input: ",pd.DataFrame(encoder_input_data).shape)

In [ ]:
# pd.DataFrame(encoder_input_data).head()

In [ ]:
# pd.DataFrame(decoder_input_data).head()

In [ ]:
# embedding_matrix = np.zeros((len(word2idx) + 1, 50))
# print("embedding_matrix shape: ",embedding_matrix.shape)

In [ ]:
# for word, i in word2idx.items():
#     embedding_vector = embeddings_index.get(word.lower())
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
# pd.DataFrame(embedding_matrix).shape

In [ ]:
# pd.DataFrame(embedding_matrix)

In [ ]:
# from tensorflow.keras.layers import Embedding
# embedding_layer = Embedding(len(word2idx) +1,
#                             50,
#                             weights=[embedding_matrix],
#                             input_length=max_encoder_seq_length,
#                             trainable=False)

In [ ]:
# encoder_input_data = np.zeros(
#     (len(input_seq), max_encoder_seq_length),
#     dtype='float32')

# decoder_input_data = np.zeros(
#     (len(target_seq), max_decoder_seq_length),
#     dtype='float32')

# decoder_target_data = np.zeros(
#     (len(target_seq), max_decoder_seq_length),
#     dtype='float32')

# for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
#     for timestep, token in enumerate(input_doc.split()):
#         encoder_input_data[line,timestep] =  embedding_matrix[input_seq[line][timestep]]
#     for timestep, token in enumerate(target_doc.split()):
#         decoder_input_data[line,timestep] = embedding_matrix[target_seq[line][timestep]]
#         if timestep > 0:
#             decoder_target_data[line, timestep - 1] = embedding_matrix[target_seq[line][timestep]]


In [ ]:
# decoder_output_data = np.zeros(shape=
#                                (len(target_seq), max_decoder_seq_length, len(word2idx)+1), 
#                                dtype="float32")
# for i, seqs in enumerate(decoder_input_data):
#     for j, seq in enumerate(seqs):
#         if j > 0:
#              decoder_output_data[i][j-1][seq] = 1.
# print(decoder_output_data.shape)

In [ ]:
# from tensorflow import keras
# from tensorflow.keras.layers import Input, LSTM, Dense
# from tensorflow.keras.models import Model
# #Dimensionality
# dimensionality = 256
# #The batch size and number of epochs
# batch_size = 10
# epochs = 150


# #Encoder
# encoder_inputs = Input(shape=(max_encoder_seq_length,))
# embedded_encoder_inputs = embedding_layer(encoder_inputs)
# encoder_lstm = LSTM(300, return_state=True)
# encoder_outputs, state_hidden, state_cell = encoder_lstm(embedded_encoder_inputs)
# encoder_states = [state_hidden, state_cell]


# #Decoder
# decoder_inputs = Input(shape=(max_decoder_seq_length,))
# embeded_decoder_inputs = Embedding(len(word2idx)+1 ,
#                             50,
#                             weights=[embedding_matrix],
#                             input_length=max_decoder_seq_length,
#                             trainable=False)(decoder_inputs)
                            
# decoder_lstm = LSTM(300, return_sequences=True, return_state=True)
# decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(embeded_decoder_inputs, initial_state=encoder_states)

# decoder_dense = Dense(len(word2idx)+1, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)
# embeded_decoder_inputs.shape

In [ ]:
# from tensorflow import keras
# from tensorflow.keras.layers import Input, LSTM, Dense
# from tensorflow.keras.models import Model

# #Model
# training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# from tensorflow.keras.utils import plot_model
# #plot_model(training_model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)
# training_model.summary()

In [ ]:
# #Compiling
# training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
# #Training
# training_model.fit([encoder_input_data, decoder_input_data], decoder_output_data, batch_size = 10, epochs = 5, validation_split = 0.2)
# training_model.save('training_model.h5')

In [ ]:
# encoder_input_data

In [ ]:
# decoder_input_data.shape

In [ ]:
# from tensorflow.keras.models import load_model
# training_model = load_model('training_model.h5')
# encoder_inputs = training_model.input[0]
# encoder_outputs, state_h_enc, state_c_enc = training_model.layers[4].output
# encoder_states = [state_h_enc, state_c_enc]
# encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
# latent_dim = 300


# decoder_state_input_hidden = Input(shape=(latent_dim,))
# decoder_state_input_cell = Input(shape=(latent_dim,))
# decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]

# decoder_outputs, state_hidden, state_cell = decoder_lstm(embeded_decoder_inputs, initial_state=decoder_states_inputs)
# decoder_states = [state_hidden, state_cell]
# decoder_outputs = decoder_dense(decoder_outputs)
# decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
# decoder_model.summary()

In [ ]:

# def decode_response(test_input):
#     #Getting the output states to pass into the decoder
#     states_value = encoder_model.predict(test_input)
#     #Generating empty target sequence of length 1
#     target_seq = np.zeros((1, max_decoder_seq_length))
#     #Setting the first token of target sequence with the start token
#     target_seq[0 , -1]= word2idx["end"]
#     #A variable to store our response word by word
#     answer = []
#     stop_condition = False
#     lenOfSentence = 1;
    
#     while not stop_condition:
   
#         output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq , states_value])
   
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         print(output_tokens[0,-1,:])
#         if(sampled_token_index == 0):
#             sampled_token=""
#         else:
#             sampled_token = idx2word[sampled_token_index]

#         answer.append(sampled_token)

#         if (sampled_token == '<END> ' or lenOfSentence > max_decoder_seq_length):
#             stop_condition = True

#         target_seq = np.zeros((1, max_decoder_seq_length))
#         target_seq[0,-1] = word2idx["end"]

#         states_value = [hidden_state, cell_state]
#         lenOfSentence +=1
#     return decoded_sentence
    

In [ ]:
# temp = np.zeros(shape = (1 , ))
# temp = tokenizer.texts_to_sequences([" hi how are you"])
# pd.DataFrame(temp)
# temp = pad_sequences(temp, maxlen=20, dtype='int32', padding='post', truncating='post')
# print(temp)
# target_seq = np.zeros((1, 1 , 300))
#     #Setting the first token of target sequence with the start token
# target_seq[0, 0, word2idx['end']] = 1
#     #A variable to store our response word by word
# decoded_sentence = ''
    
# stop_condition = False
# decode_response(temp)

In [ ]:
# class ChatBot:
#   negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")
#   exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
#     #Method to start the conversation
#   def start_chat(self):
#     user_response = input("Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?\n")
    
#     if user_response in self.negative_responses:
#       print("Ok, have a great day!")
#       return
#     self.chat(user_response)#Method to handle the conversation
#   def chat(self, reply):
#     while not self.make_exit(reply):
#       reply = input(self.generate_response(reply)+"\n")
    
#   #Method to convert user input into a matrix
#   def string_to_matrix(self, user_input):
#     tokens = clean_text(user_input).split()
#     user_input_matrix = np.zeros(
#       (1, max_encoder_seq_length),
#       dtype='float32')
#     user_input_embedded = pad_sequences(user_input, maxlen=max_encoder_seq_length, dtype='int32', padding='post', truncating='post')
#     user_input_matrix = tokenizer.texts_to_sequences([user_input_embedded])
#     return user_input_matrix
  
#   #Method that will create a response using seq2seq model we built
#   def generate_response(self, user_input):
#     input_matrix = self.string_to_matrix(user_input)
#     chatbot_response = decode_response(input_matrix)
#     #Remove <START> and <END> tokens from chatbot_response
#     chatbot_response = chatbot_response.replace("<START>",'')
#     chatbot_response = chatbot_response.replace("<END>",'')
#     return chatbot_response#Method to check for exit commands
#   def make_exit(self, reply):
#     for exit_command in self.exit_commands:
#       if exit_command in reply:
#         print("Ok, have a great day!")
#         return True
#     return False
  
# chatbot = ChatBot()
# chatbot.start_chat()

In [ ]:
# embeddings_index = {}
# f = open(os.path.join(path, 'glove.6B.50d.txt'))
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()